<center><img src="image.png" width=500></center>
<p>

You've recently started a new position as a Data Engineer at an energy company. Previously, analysts on other teams had to manually retrieve and clean data every quarter to understand changes in the sales and capability of different energy types. This process normally took days and was something that most analytsts dreaded. Your job is to automate this process by building a data pipeline. You'll write this data pipeline to pull data each month, helping to provide more rapid insights and free up time for your data consumers.

You will achieve this using the `pandas` library and its powerful parsing features. You'll be working with two raw files; `electricity_sales.csv` and `electricity_capability_nested.json`. 
    
Below, you'll find a data dictionary for the `electricity_sales.csv` dataset, which you'll be transforming in just a bit. Good luck!

| Field | Data Type |
| :---- | :-------: |
| period  | `str`        |
| stateid | `str` |
| stateDescription | `str` |
| sectorid | `str` |
| sectorName | `str` |
| price | `float` |
| price-units | `str` |

In [2]:
# importing libraries
import pandas as pd
import json

In [ ]:
# extracting data
def extract_tabular_data(file_path: str):
    """Extract data from a tabular file_format, with pandas."""
    return pd.read_csv(file_path)


In [ ]:
# extracting json data
def extract_json_data(file_path):
    """Extract and flatten data from a JSON file."""
    with open(file_path) as f:
        data = json.load(f)
    return pd.json_normalize(data)

In [ ]:
def transform_electricity_sales_data(raw_data: pd.DataFrame):
    """
    Transform electricity sales to find the total amount of electricity sold
    in the residential and transportation sectors.
    
    To transform the electricity sales data, you'll need to do the following:
    - Drop any records with NA values in the `price` column. Do this inplace.
    - Only keep records with a `sectorName` of "residential" or "transportation".
    - Create a `month` column using the first 4 characters of the values in `period`.
    - Create a `year` column using the last 2 characters of the values in `period`.
    - Return the transformed `DataFrame`, keeping only the columns `year`, `month`, `stateid`, `price` and `price-units`.
    """


    # It's better to work on a copy to avoid modifying the original DataFrame unexpectedly.
    transformed_df = raw_data.copy()

    # The 'price' column is of object type due to empty strings. Convert to numeric.
    # `errors='coerce'` will turn non-numeric values (like empty strings) into NaN.
    transformed_df['price'] = pd.to_numeric(transformed_df['price'], errors='coerce')

    # Drop records with NA in 'price'
    transformed_df.dropna(subset=['price'], inplace=True)

    # Filter for 'residential' or 'transportation' sectors
    transformed_df = transformed_df[transformed_df['sectorName'].isin(['residential', 'transportation'])]

    # Create 'year' and 'month' columns from the 'period'
    transformed_df['year'] = pd.to_datetime(transformed_df['period']).dt.year
    transformed_df['month'] = pd.to_datetime(transformed_df['period']).dt.month

    # Select and return the required columns
    return transformed_df[['year', 'month', 'stateid', 'price', 'price-units']]



In [ ]:
def load(dataframe: pd.DataFrame, file_path: str):
    """Load a DataFrame to a file in either CSV or Parquet format."""
    if file_path.endswith('.csv'):
        dataframe.to_csv(file_path, index=False)
    elif file_path.endswith('.parquet'):
        dataframe.to_parquet(file_path, index=False)
    else:
        raise ValueError("Unsupported file format. Please use CSV or Parquet.")

In [ ]:
# Ready for the moment of truth? It's time to test the functions that you wrote!
raw_electricity_capability_df = extract_json_data("electricity_capability_nested.json")
raw_electricity_sales_df = extract_tabular_data("electricity_sales.csv")

cleaned_electricity_sales_df = transform_electricity_sales_data(raw_electricity_sales_df)

load(raw_electricity_capability_df, "output/loaded__electricity_capability.parquet")
load(cleaned_electricity_sales_df, "output/loaded__electricity_sales.csv")